In [1]:
#pip install unidecode

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: unidecode in c:\users\jquispe\appdata\local\programs\python\python311\lib\site-packages (1.3.7)




[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu110 (from versions: 2.0.0, 2.0.0+cpu, 2.0.0+cu117, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cu117, 2.0.1+cu118, 2.1.0, 2.1.0+cpu, 2.1.0+cu118, 2.1.0+cu121)
ERROR: No matching distribution found for torch==1.7.1+cu110

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


IMPORTACIÓN DE LIBRERIAS

In [2]:
import pyodbc
import torch
from scipy.spatial.distance import cosine
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from unidecode import unidecode
from datetime import datetime
from pymongo import MongoClient
import re
import pyodbc
import numpy as np

c:\Users\jquispe\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Conexión BDD RELACIONAL

In [3]:
# Establece la cadena de conexión a tu servidor SQL Server
server = 'BFICHASIST23'
database = 'laboratorio'
username = 'sa'
password = '1719895086'

conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};TIMEOUT=30')

In [3]:
# Ejecutamos el query para realizar la consulta a la base
base_med = "SELECT id_medico, apellido_paterno, apellido_materno, nombres, numero_identificacion, id_especialidad FROM dbo.medicos"

#  Se guarda en un df 
sql_data = pd.read_sql_query(base_med, conn)

#Se guarda el df en una nueva versión para no afectar la primera tabla
sql_data2 = sql_data

#Se concatena el nombre y apellidos para poder ejecutar el modelo
sql_data2['NOMBRE_COMPLETO_SQL'] = sql_data['apellido_paterno'] + ' ' + sql_data['apellido_materno'] + ' ' + sql_data['nombres']
sql_data2

C:\Users\jquispe\AppData\Local\Temp\ipykernel_8860\706134697.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_data = pd.read_sql_query(base_med, conn)


,id_medico,apellido_paterno,apellido_materno,nombres,numero_identificacion,id_especialidad,NOMBRE_COMPLETO_SQL
0,1,ROMERO,HERRERA,DENISE ELCIRA,0920116530,6,ROMERO HERRERA DENISE ELCIRA
1,2,FLORES,YANANGOMEZ,DIANA ALEXANDRA,1103532808,11,FLORES YANANGOMEZ DIANA ALEXANDRA
2,3,NAVEDA,PEREZ,DIANA MARICELA,1804125001,4,NAVEDA PEREZ DIANA MARICELA
3,4,COELLO,AGUILAR,DIANA MARLENE,0702216052,15,COELLO AGUILAR DIANA MARLENE
4,5,JUAREZ,CARPIO,DIEGO EDMUNDO,0701072472,2,JUAREZ CARPIO DIEGO EDMUNDO
...,...,...,...,...,...,...,...
3113,3118,MATAMOROS,ORELLANA,KAREN LIZETTE,0704012392,23,MATAMOROS ORELLANA KAREN LIZETTE
3114,3119,MIRANDA,GARCES,MARIA DE LOURDES,1714038211,23,MIRANDA GARCES MARIA DE LOURDES
3115,3120,PILLAJO,BALLADARES,GLORIA JANETH,1712510419,23,PILLAJO BALLADARES GLORIA JANETH
3116,3121,PEZ,SANCHEZ,MARTHA CECILIA,1703898567,12,PEZ SANCHEZ MARTHA CECILIA


In [4]:
# Mostrar el DataFrame resultante
sql_data2 = sql_data2.iloc[:,[0,6,5,4]]
sql_data2

,id_medico,NOMBRE_COMPLETO_SQL,id_especialidad,numero_identificacion
0,1,ROMERO HERRERA DENISE ELCIRA,6,0920116530
1,2,FLORES YANANGOMEZ DIANA ALEXANDRA,11,1103532808
2,3,NAVEDA PEREZ DIANA MARICELA,4,1804125001
3,4,COELLO AGUILAR DIANA MARLENE,15,0702216052
4,5,JUAREZ CARPIO DIEGO EDMUNDO,2,0701072472
...,...,...,...,...
3113,3118,MATAMOROS ORELLANA KAREN LIZETTE,23,0704012392
3114,3119,MIRANDA GARCES MARIA DE LOURDES,23,1714038211
3115,3120,PILLAJO BALLADARES GLORIA JANETH,23,1712510419
3116,3121,PEZ SANCHEZ MARTHA CECILIA,12,1703898567


In [5]:
# Ejecutamos el query para realizar la consulta a la base
especialidad = "select id_especialidad ID_ESPECIALIDAD, nombre_especialidad ESPECIALIDAD from dbo.especialidades"
# Se guarda en un df 
especialidad = pd.read_sql_query(especialidad, conn)
especialidad.head()

C:\Users\jquispe\AppData\Local\Temp\ipykernel_8860\804103328.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  especialidad = pd.read_sql_query(especialidad, conn)


,ID_ESPECIALIDAD,ESPECIALIDAD
0,1,ALERGOLOGIA
1,2,CARDIOLOGIA
2,3,CIRUGIA CARDIOVASCULAR
3,4,CIRUGIA GENERAL
4,5,CIRUGIA ORAL Y MAXILOFACIAL


CARGA DE DATOS MEDICOS NUEVOS

In [6]:
# Lee el archivo Excel en un DataFrame
med_nuevos = pd.read_excel('C:/Users/jquispe/Desktop/JACQUE/MAESTRÍA/tesis/NLP/INSUMO/ALTAS MED_1.xlsm', skiprows=1)
#Para no afectar la tabla inicial generamos una copia
med_nuevos2 = med_nuevos
#Seleccionamos las columnas con las que se va a trabajar
med_nuevos2 = med_nuevos2.iloc[:, :-7].copy()
#Se concatena el nombre y apellidos para poder ejecutar el modelo
med_nuevos2['NOMBRE_COMPLETO_EXC'] = med_nuevos2['APELLIDO_PAT'] + ' ' + med_nuevos2['APELLIDO_MAT'] + ' ' + med_nuevos2['NOMBRES']
med_nuevos2.head()

c:\Users\jquispe\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,ESPECIALIDAD,DIRECCION,NOMBRE_COMPLETO_EXC
0,2581014,ALVARADO,CORDOVA,MARCY RODELY,702884826,1975-01-01,MARCY.ALVARADO@HOTMAIL.COM,993214598,1,MEDICINA GENERAL,CALLE 2 N 1256 Y CALLE 6,ALVARADO CORDOVA MARCY RODELY
1,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995-02-14,RITA.BRAVO@GMAIL.COM,985214566,3,GINECOLOGIA Y OBSTETRICIA,CALLE 3 N 1243 Y CALLE 7,BRAVO QUIJANO RITA ANNABEL
2,2581018,CORDOVA,PALADINES,JENNY ELIZABETH,1103175459,1980-12-25,JENNY.CORDOVA@GMAIL.COM,965874122,1,PEDIATRIA,CALLE 5 S 8554 Y CALLE 9,CORDOVA PALADINES JENNY ELIZABETH
3,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970-08-10,WILSON.LEMA@GMAIL.COM,963574122,3,MEDICINA INTERNA,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE
4,2581022,MEDINA,DIAZ,ANABEL EMMANUELLA,1804027710,1991-08-06,ANABEL.MEDINA@GMAIL.COM,996324410,1,MEDICINA GENERAL,AVENIDA 9 N 45564 Y CALLE 1,MEDINA DIAZ ANABEL EMMANUELLA


In [7]:
# Realiza la fusión (merge) de los DataFrames para obtener el código de especialidad
med_nuevos2 = med_nuevos2.merge(especialidad, on='ESPECIALIDAD', how='left')
# Renombra la columna 'Codigo' a 'Especialidad'
med_nuevos2 = med_nuevos2.rename(columns={'id_especialidad': 'cod_Espe'})
# Eliminar una columna específica del DataFrame
med_nuevos2 = med_nuevos2.drop('ESPECIALIDAD', axis=1)
med_nuevos2

,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD
0,2581014,ALVARADO,CORDOVA,MARCY RODELY,702884826,1975-01-01,MARCY.ALVARADO@HOTMAIL.COM,993214598,1,CALLE 2 N 1256 Y CALLE 6,ALVARADO CORDOVA MARCY RODELY,23
1,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995-02-14,RITA.BRAVO@GMAIL.COM,985214566,3,CALLE 3 N 1243 Y CALLE 7,BRAVO QUIJANO RITA ANNABEL,15
2,2581018,CORDOVA,PALADINES,JENNY ELIZABETH,1103175459,1980-12-25,JENNY.CORDOVA@GMAIL.COM,965874122,1,CALLE 5 S 8554 Y CALLE 9,CORDOVA PALADINES JENNY ELIZABETH,37
3,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970-08-10,WILSON.LEMA@GMAIL.COM,963574122,3,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE,22
4,2581022,MEDINA,DIAZ,ANABEL EMMANUELLA,1804027710,1991-08-06,ANABEL.MEDINA@GMAIL.COM,996324410,1,AVENIDA 9 N 45564 Y CALLE 1,MEDINA DIAZ ANABEL EMMANUELLA,23
5,2581024,PAEZ,SANCHEZ,MARTHA CECILIA,1703898567,1992-09-18,MARTHA.PAEZ@GMAIL.COM,987444112,3,AVENIDA 14 S 85541,PAEZ SANCHEZ MARTHA CECILIA,12
6,2581026,PIN,GUADAMUD,JINMY ALEXANDER,1307584522,1959-11-23,JINMY.PIN@GMAIL.COM,996321447,1,AVENIDA 18 N 526 Y CALLE 632,PIN GUADAMUD JINMY ALEXANDER,15
7,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970-08-10,WILSON.LEMA@GMAIL.COM,963574122,3,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE,23
8,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970-08-10,WILSON.LEMA@GMAIL.COM,963574122,3,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE,23
9,2581027,REDROVAN,TENESACA,FRANCISCA EDITA,701791634,1963-06-29,EDITAFRANCISCA.REDROVANTENESACA@HOTMAIL.COM,971930482,2,CALLE 4179 NRO 6298 Y CALLE 736,REDROVAN TENESACA FRANCISCA EDITA,20


In [8]:
med_nuevos2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CODCUP               13 non-null     int64         
 1   APELLIDO_PAT         13 non-null     object        
 2   APELLIDO_MAT         13 non-null     object        
 3   NOMBRES              13 non-null     object        
 4   CI                   13 non-null     int64         
 5   FEC_NAC              13 non-null     datetime64[ns]
 6   EMAIL                13 non-null     object        
 7   TEL                  13 non-null     int64         
 8   Q_FICO               13 non-null     int64         
 9   DIRECCION            13 non-null     object        
 10  NOMBRE_COMPLETO_EXC  13 non-null     object        
 11  ID_ESPECIALIDAD      13 non-null     int64         
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 1.3+ KB


TRATAMIENTO DE DATOS

In [9]:
# Función para eliminar tildes y convertir "ñ" en "n" en una cadena
def quitar_tildes_y_n(cadena):
    if isinstance(cadena, str):
        cadena_sin_tildes = unidecode(cadena)
        cadena_sin_n = cadena_sin_tildes.replace('ñ', 'n')
        return cadena_sin_n
    else:
        return cadena
    
# Aplicar la función eliminar tildes y ñ solo a las columnas de tipo objeto (texto)
for columna in med_nuevos2.select_dtypes(include=['object']).columns:
    med_nuevos2[columna] = med_nuevos2[columna].apply(quitar_tildes_y_n) # esta funcion quita las tildes y las ñ

# Eliminación de espacios en blanco
med_nuevos2[columna] = med_nuevos2[columna].str.strip()

# elimina las filas duplicadas 
med_nuevos2 = med_nuevos2.drop_duplicates() 

# Convierte en tipo texto
med_nuevos2['TEL'] = med_nuevos2['TEL'].apply(lambda x: str(x).replace('.0', ''))
med_nuevos2['Q_FICO'] = med_nuevos2['Q_FICO'].apply(lambda x: str(x).replace('.0', ''))
med_nuevos2['CI'] = med_nuevos2['CI'].astype(str)

# Función para que la cedula y celular tenga la estructura de 10 digitos
def convertir_a_texto(valor):
    if isinstance(valor, str):
        valor_texto = str(valor)
        if len(valor_texto) == 9:
            valor_texto = '0' + valor_texto
        return valor_texto
    else:
        return str(valor)
    
# Aplicar la función solo a las columnas "ci" y "celular"
med_nuevos2['TEL'] = med_nuevos2['TEL'].apply(convertir_a_texto)
med_nuevos2['CI'] = med_nuevos2['CI'].apply(convertir_a_texto)
    

# Función para eliminar caracteres especiales de una cadena
def eliminar_caracteres_especiales(cadena):
    if isinstance(cadena, str):
        return re.sub(r'[^\w\s]', '', cadena)
    else:
        return cadena

# Obtener las columnas de texto (excepto la columna de correo)
columnas_texto = [col for col in med_nuevos2.select_dtypes(include=['object']).columns if col != 'EMAIL']

# Iterar a través de las columnas de texto y aplicar la función
for columna in columnas_texto:
    med_nuevos2[columna] = med_nuevos2[columna].apply(eliminar_caracteres_especiales)

# convierte en mayusculas absolutamente todo el texto 
med_nuevos2 = med_nuevos2.applymap(lambda x: x.upper() if isinstance(x, str) else x) 


# Visualizar los cambios
med_nuevos2

C:\Users\jquispe\AppData\Local\Temp\ipykernel_8860\2893761507.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  med_nuevos2['TEL'] = med_nuevos2['TEL'].apply(lambda x: str(x).replace('.0', ''))
C:\Users\jquispe\AppData\Local\Temp\ipykernel_8860\2893761507.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  med_nuevos2['Q_FICO'] = med_nuevos2['Q_FICO'].apply(lambda x: str(x).replace('.0', ''))
C:\Users\jquispe\AppData\Local\Temp\ipykernel_8860\2893761507.py:23: SettingWithCopyWarning: 
A value is trying 

,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD
0,2581014,ALVARADO,CORDOVA,MARCY RODELY,0702884826,1975-01-01,MARCY.ALVARADO@HOTMAIL.COM,0993214598,1,CALLE 2 N 1256 Y CALLE 6,ALVARADO CORDOVA MARCY RODELY,23
1,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995-02-14,RITA.BRAVO@GMAIL.COM,0985214566,3,CALLE 3 N 1243 Y CALLE 7,BRAVO QUIJANO RITA ANNABEL,15
2,2581018,CORDOVA,PALADINES,JENNY ELIZABETH,1103175459,1980-12-25,JENNY.CORDOVA@GMAIL.COM,0965874122,1,CALLE 5 S 8554 Y CALLE 9,CORDOVA PALADINES JENNY ELIZABETH,37
3,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970-08-10,WILSON.LEMA@GMAIL.COM,0963574122,3,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE,22
4,2581022,MEDINA,DIAZ,ANABEL EMMANUELLA,1804027710,1991-08-06,ANABEL.MEDINA@GMAIL.COM,0996324410,1,AVENIDA 9 N 45564 Y CALLE 1,MEDINA DIAZ ANABEL EMMANUELLA,23
5,2581024,PAEZ,SANCHEZ,MARTHA CECILIA,1703898567,1992-09-18,MARTHA.PAEZ@GMAIL.COM,0987444112,3,AVENIDA 14 S 85541,PAEZ SANCHEZ MARTHA CECILIA,12
6,2581026,PIN,GUADAMUD,JINMY ALEXANDER,1307584522,1959-11-23,JINMY.PIN@GMAIL.COM,0996321447,1,AVENIDA 18 N 526 Y CALLE 632,PIN GUADAMUD JINMY ALEXANDER,15
7,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970-08-10,WILSON.LEMA@GMAIL.COM,0963574122,3,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE,23
9,2581027,REDROVAN,TENESACA,FRANCISCA EDITA,0701791634,1963-06-29,EDITAFRANCISCA.REDROVANTENESACA@HOTMAIL.COM,0971930482,2,CALLE 4179 NRO 6298 Y CALLE 736,REDROVAN TENESACA FRANCISCA EDITA,20
10,2581029,SALAS,DAU,FLORA SORAYA,0913837910,1968-05-17,FLORASORAYA.SALASDAU@HOTMAIL.COM,0961882647,2,CALLE 2590 NRO 1456 Y CALLE 3034,SALAS DAU FLORA SORAYA,16


In [10]:
#Fecha de nacimiento a texto y con el formato adecuado para cargar a las bases
med_nuevos2['FEC_NAC'] = med_nuevos2['FEC_NAC'].dt.strftime('%Y/%m/%d')

med_nuevos2


,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD
0,2581014,ALVARADO,CORDOVA,MARCY RODELY,0702884826,1975/01/01,MARCY.ALVARADO@HOTMAIL.COM,0993214598,1,CALLE 2 N 1256 Y CALLE 6,ALVARADO CORDOVA MARCY RODELY,23
1,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995/02/14,RITA.BRAVO@GMAIL.COM,0985214566,3,CALLE 3 N 1243 Y CALLE 7,BRAVO QUIJANO RITA ANNABEL,15
2,2581018,CORDOVA,PALADINES,JENNY ELIZABETH,1103175459,1980/12/25,JENNY.CORDOVA@GMAIL.COM,0965874122,1,CALLE 5 S 8554 Y CALLE 9,CORDOVA PALADINES JENNY ELIZABETH,37
3,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970/08/10,WILSON.LEMA@GMAIL.COM,0963574122,3,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE,22
4,2581022,MEDINA,DIAZ,ANABEL EMMANUELLA,1804027710,1991/08/06,ANABEL.MEDINA@GMAIL.COM,0996324410,1,AVENIDA 9 N 45564 Y CALLE 1,MEDINA DIAZ ANABEL EMMANUELLA,23
5,2581024,PAEZ,SANCHEZ,MARTHA CECILIA,1703898567,1992/09/18,MARTHA.PAEZ@GMAIL.COM,0987444112,3,AVENIDA 14 S 85541,PAEZ SANCHEZ MARTHA CECILIA,12
6,2581026,PIN,GUADAMUD,JINMY ALEXANDER,1307584522,1959/11/23,JINMY.PIN@GMAIL.COM,0996321447,1,AVENIDA 18 N 526 Y CALLE 632,PIN GUADAMUD JINMY ALEXANDER,15
7,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970/08/10,WILSON.LEMA@GMAIL.COM,0963574122,3,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE,23
9,2581027,REDROVAN,TENESACA,FRANCISCA EDITA,0701791634,1963/06/29,EDITAFRANCISCA.REDROVANTENESACA@HOTMAIL.COM,0971930482,2,CALLE 4179 NRO 6298 Y CALLE 736,REDROVAN TENESACA FRANCISCA EDITA,20
10,2581029,SALAS,DAU,FLORA SORAYA,0913837910,1968/05/17,FLORASORAYA.SALASDAU@HOTMAIL.COM,0961882647,2,CALLE 2590 NRO 1456 Y CALLE 3034,SALAS DAU FLORA SORAYA,16


Algoritmo de verificación de cédulas

In [11]:
def validar_cedula(cedula):
    # Verificar que la cédula tenga 10 dígitos
    if len(cedula) != 10:
        return False
    
    # Verificar que los primeros dos dígitos sean válidos
    provincia = int(cedula[:2])
    if provincia < 1 or provincia > 24:
        return False
    
    # Verificar el tercer dígito (dígito de control)
    tercer_digito = int(cedula[2])
    if tercer_digito < 0 or tercer_digito > 5:
        return False
    
    # Verificar el último dígito (dígito de verificación)
    verificacion = int(cedula[9])
    
    # Calcular el dígito de verificación esperado
    coeficientes = [2, 1, 2, 1, 2, 1, 2, 1, 2]
    suma = 0
    for i in range(9):
        digito = int(cedula[i])
        producto = digito * coeficientes[i]
        if producto > 9:
            producto -= 9
        suma += producto
    
    verificacion_esperada = 10 - (suma % 10)
    if verificacion_esperada == 10:
        verificacion_esperada = 0
    
    # Comparar el dígito de verificación calculado con el proporcionado
    return verificacion == verificacion_esperada

# Aplicacion de la funcion
med_nuevos2['verif_ced'] = med_nuevos2['CI'].apply(validar_cedula)
med_nuevos2


,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD,verif_ced
0,2581014,ALVARADO,CORDOVA,MARCY RODELY,0702884826,1975/01/01,MARCY.ALVARADO@HOTMAIL.COM,0993214598,1,CALLE 2 N 1256 Y CALLE 6,ALVARADO CORDOVA MARCY RODELY,23,True
1,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995/02/14,RITA.BRAVO@GMAIL.COM,0985214566,3,CALLE 3 N 1243 Y CALLE 7,BRAVO QUIJANO RITA ANNABEL,15,True
2,2581018,CORDOVA,PALADINES,JENNY ELIZABETH,1103175459,1980/12/25,JENNY.CORDOVA@GMAIL.COM,0965874122,1,CALLE 5 S 8554 Y CALLE 9,CORDOVA PALADINES JENNY ELIZABETH,37,True
3,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970/08/10,WILSON.LEMA@GMAIL.COM,0963574122,3,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE,22,True
4,2581022,MEDINA,DIAZ,ANABEL EMMANUELLA,1804027710,1991/08/06,ANABEL.MEDINA@GMAIL.COM,0996324410,1,AVENIDA 9 N 45564 Y CALLE 1,MEDINA DIAZ ANABEL EMMANUELLA,23,True
5,2581024,PAEZ,SANCHEZ,MARTHA CECILIA,1703898567,1992/09/18,MARTHA.PAEZ@GMAIL.COM,0987444112,3,AVENIDA 14 S 85541,PAEZ SANCHEZ MARTHA CECILIA,12,True
6,2581026,PIN,GUADAMUD,JINMY ALEXANDER,1307584522,1959/11/23,JINMY.PIN@GMAIL.COM,0996321447,1,AVENIDA 18 N 526 Y CALLE 632,PIN GUADAMUD JINMY ALEXANDER,15,True
7,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970/08/10,WILSON.LEMA@GMAIL.COM,0963574122,3,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE,23,True
9,2581027,REDROVAN,TENESACA,FRANCISCA EDITA,0701791634,1963/06/29,EDITAFRANCISCA.REDROVANTENESACA@HOTMAIL.COM,0971930482,2,CALLE 4179 NRO 6298 Y CALLE 736,REDROVAN TENESACA FRANCISCA EDITA,20,True
10,2581029,SALAS,DAU,FLORA SORAYA,0913837910,1968/05/17,FLORASORAYA.SALASDAU@HOTMAIL.COM,0961882647,2,CALLE 2590 NRO 1456 Y CALLE 3034,SALAS DAU FLORA SORAYA,16,True


MODELO DE DETECCION DE DUPLICADOS

MODELO FUZZY RATIO

In [27]:
# Crear una lista para almacenar las coincidencias
coincidencias = []

# Iterar a través de los nombres en el archivo Excel
for index_excel, row_excel in med_nuevos2.iterrows():
    nombre_excel = row_excel['NOMBRE_COMPLETO_EXC']
    mejor_coincidencia = None
    mejor_porcentaje = 0

    for index_db, row_db in sql_data2.iterrows():
        nombre_db = row_db['NOMBRE_COMPLETO_SQL']

        # Calcular la similitud entre los nombres y apellidos
        similitud_nombre = fuzz.ratio(nombre_excel.lower(), nombre_db.lower())

        # Si la similitud está por encima del umbral y es mejor que la anterior, actualizar la mejor coincidencia
        if similitud_nombre > mejor_porcentaje:
            mejor_coincidencia = pd.concat([row_excel, row_db])
            mejor_coincidencia['Porcentaje de Similitud'] = similitud_nombre
            mejor_porcentaje = similitud_nombre

    # Agregar la mejor coincidencia a la lista de coincidencias
    if mejor_coincidencia is not None:
        coincidencias.append(mejor_coincidencia)

# Crear un DataFrame con las mejores coincidencias
coincidencias_df1 = pd.DataFrame(coincidencias)

# Función para asignar "Medico en Base" o "Medico Nuevo" en función de la similitud
def asignar_medico(row):
    return "MEDICO EN BASE" if row['Porcentaje de Similitud'] > 79 else "MEDICO NUEVO"

# Aplicar la función a cada fila y crear una nueva columna "OBSERVACION"
coincidencias_df1['OBSERVACION'] = coincidencias_df1.apply(asignar_medico, axis=1)

# Validación de cédulas
coincidencias_df1['VAL_CED'] = coincidencias_df1['CI'] == coincidencias_df1['numero_identificacion']

# Validación de especialidad
coincidencias_df1['VAL_ESPE'] = coincidencias_df1['ID_ESPECIALIDAD'] == coincidencias_df1['id_especialidad']

coincidencias_df1 = coincidencias_df1.iloc[:,[0,4,10,11,12,13,14,15,16,17,18,19,20]]

coincidencias_df1.head()

,CODCUP,CI,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD,verif_ced,id_medico,NOMBRE_COMPLETO_SQL,id_especialidad,numero_identificacion,Porcentaje de Similitud,OBSERVACION,VAL_CED,VAL_ESPE
0,2581014,0702884826,ALVARADO CORDOVA MARCY RODELY,23,True,14,ALVARADO CEDEÑO DORA VILMA,34,0906016357,61,MEDICO NUEVO,False,False
1,2581016,1307604478,BRAVO QUIJANO RITA ANNABEL,15,True,3122,BRAVO QUIJANO RITA ANNABEL,15,1307604478,100,MEDICO EN BASE,True,True
2,2581018,1103175459,CORDOVA PALADINES JENNY ELIZABETH,37,True,2232,DIAZ AYALA GENNY ELIZABETH,7,1001731213,70,MEDICO NUEVO,False,False
3,2581020,1709036824,LEMA LEMA WILSON ENRIQUE,22,True,1502,VILELA SILVA JULIO ENRIQUE,39,0701317778,67,MEDICO NUEVO,False,False
4,2581022,1804027710,MEDINA DIAZ ANABEL EMMANUELLA,23,True,442,GARCIA MEDINA SANDRA ELENA,1,1802266435,57,MEDICO NUEVO,False,False


MODELO FUZZY PARTIAL RATIO

In [28]:
# Crear una lista para almacenar las coincidencias
coincidencias = []

# Iterar a través de los nombres en el archivo Excel
for index_excel, row_excel in med_nuevos2.iterrows():
    nombre_excel = row_excel['NOMBRE_COMPLETO_EXC']
    mejor_coincidencia = None
    mejor_porcentaje = 0

    for index_db, row_db in sql_data2.iterrows():
        nombre_db = row_db['NOMBRE_COMPLETO_SQL']

        # Calcular la similitud entre los nombres y apellidos
        similitud_nombre = fuzz.partial_ratio(nombre_excel.lower(), nombre_db.lower())

        # Si la similitud está por encima del umbral y es mejor que la anterior, actualizar la mejor coincidencia
        if similitud_nombre > mejor_porcentaje:
            mejor_coincidencia = pd.concat([row_excel, row_db])
            mejor_coincidencia['Porcentaje de Similitud'] = similitud_nombre
            mejor_porcentaje = similitud_nombre

    # Agregar la mejor coincidencia a la lista de coincidencias
    if mejor_coincidencia is not None:
        coincidencias.append(mejor_coincidencia)

# Crear un DataFrame con las mejores coincidencias
coincidencias_df2 = pd.DataFrame(coincidencias)

# Función para asignar "Medico en Base" o "Medico Nuevo" en función de la similitud
def asignar_medico(row):
    return "MEDICO EN BASE" if row['Porcentaje de Similitud'] > 79 else "MEDICO NUEVO"

# Aplicar la función a cada fila y crear una nueva columna "OBSERVACION"
coincidencias_df2['OBSERVACION'] = coincidencias_df2.apply(asignar_medico, axis=1)

# Validación de cédulas
coincidencias_df2['VAL_CED'] = coincidencias_df2['CI'] == coincidencias_df2['numero_identificacion']

# Validación de especialidad
coincidencias_df2['VAL_ESP'] = coincidencias_df2['ID_ESPECIALIDAD'] == coincidencias_df2['id_especialidad']

coincidencias_df2 = coincidencias_df2.iloc[:,[0,4,10,11,12,13,14,15,16,17,18,19,20]]

coincidencias_df2.head()


,CODCUP,CI,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD,verif_ced,id_medico,NOMBRE_COMPLETO_SQL,id_especialidad,numero_identificacion,Porcentaje de Similitud,OBSERVACION,VAL_CED,VAL_ESP
0,2581014,0702884826,ALVARADO CORDOVA MARCY RODELY,23,True,14,ALVARADO CEDEÑO DORA VILMA,34,0906016357,63,MEDICO NUEVO,False,False
1,2581016,1307604478,BRAVO QUIJANO RITA ANNABEL,15,True,3122,BRAVO QUIJANO RITA ANNABEL,15,1307604478,100,MEDICO EN BASE,True,True
2,2581018,1103175459,CORDOVA PALADINES JENNY ELIZABETH,37,True,2232,DIAZ AYALA GENNY ELIZABETH,7,1001731213,74,MEDICO NUEVO,False,False
3,2581020,1709036824,LEMA LEMA WILSON ENRIQUE,22,True,1502,VILELA SILVA JULIO ENRIQUE,39,0701317778,68,MEDICO NUEVO,False,False
4,2581022,1804027710,MEDINA DIAZ ANABEL EMMANUELLA,23,True,911,DIAZ SANCHEZ MARIA ANGELICA,13,1600194854,60,MEDICO NUEVO,False,False


MODELO FUZZY TOKEN SORT RATIO

In [29]:
# Crear una lista para almacenar las coincidencias
coincidencias = []

# Iterar a través de los nombres en el archivo Excel
for index_excel, row_excel in med_nuevos2.iterrows():
    nombre_excel = row_excel['NOMBRE_COMPLETO_EXC']
    mejor_coincidencia = None
    mejor_porcentaje = 0

    for index_db, row_db in sql_data2.iterrows():
        nombre_db = row_db['NOMBRE_COMPLETO_SQL']

        # Calcular la similitud entre los nombres y apellidos
        similitud_nombre = fuzz.token_sort_ratio(nombre_excel.lower(), nombre_db.lower())

        # Si la similitud está por encima del umbral y es mejor que la anterior, actualizar la mejor coincidencia
        if similitud_nombre > mejor_porcentaje:
            mejor_coincidencia = pd.concat([row_excel, row_db])
            mejor_coincidencia['Porcentaje de Similitud'] = similitud_nombre
            mejor_porcentaje = similitud_nombre

    # Agregar la mejor coincidencia a la lista de coincidencias
    if mejor_coincidencia is not None:
        coincidencias.append(mejor_coincidencia)

# Crear un DataFrame con las mejores coincidencias
coincidencias_df = pd.DataFrame(coincidencias)

# Función para asignar "Medico en Base" o "Medico Nuevo" en función de la similitud
def asignar_medico(row):
    return "MEDICO EN BASE" if row['Porcentaje de Similitud'] > 79 else "MEDICO NUEVO"

# Aplicar la función a cada fila y crear una nueva columna "OBSERVACION"
coincidencias_df['OBSERVACION'] = coincidencias_df.apply(asignar_medico, axis=1)

# Validación de cédulas
coincidencias_df['VAL_CED'] = coincidencias_df['CI'] == coincidencias_df['numero_identificacion']

# Validación de especialidad
coincidencias_df['VAL_ESP'] = coincidencias_df['ID_ESPECIALIDAD'] == coincidencias_df['id_especialidad']

coincidencias_df0= coincidencias_df
coincidencias_df0 = coincidencias_df0.iloc[:,[0,4,10,11,12,13,14,15,16,17,18,19,20]]



coincidencias_df0.head()

,CODCUP,CI,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD,verif_ced,id_medico,NOMBRE_COMPLETO_SQL,id_especialidad,numero_identificacion,Porcentaje de Similitud,OBSERVACION,VAL_CED,VAL_ESP
0,2581014,0702884826,ALVARADO CORDOVA MARCY RODELY,23,True,14,ALVARADO CEDEÑO DORA VILMA,34,0906016357,63,MEDICO NUEVO,False,False
1,2581016,1307604478,BRAVO QUIJANO RITA ANNABEL,15,True,3122,BRAVO QUIJANO RITA ANNABEL,15,1307604478,100,MEDICO EN BASE,True,True
2,2581018,1103175459,CORDOVA PALADINES JENNY ELIZABETH,37,True,786,CORDOVILLA PALOMINO JENNY FABIOLA,36,0503072597,67,MEDICO NUEVO,False,False
3,2581020,1709036824,LEMA LEMA WILSON ENRIQUE,22,True,230,LLERENA GUEVARA LUIS ENRIQUE,39,1801412691,65,MEDICO NUEVO,False,False
4,2581022,1804027710,MEDINA DIAZ ANABEL EMMANUELLA,23,True,923,CORONEL MEDINA MARIA ISABEL,30,0702720525,61,MEDICO NUEVO,False,False


MODELO FUZZY TOKEN SET RATIO

In [30]:
# Crear una lista para almacenar las coincidencias
coincidencias = []

# Iterar a través de los nombres en el archivo Excel
for index_excel, row_excel in med_nuevos2.iterrows():
    nombre_excel = row_excel['NOMBRE_COMPLETO_EXC']
    mejor_coincidencia = None
    mejor_porcentaje = 0

    for index_db, row_db in sql_data2.iterrows():
        nombre_db = row_db['NOMBRE_COMPLETO_SQL']

        # Calcular la similitud entre los nombres y apellidos
        similitud_nombre = fuzz.token_set_ratio(nombre_excel.lower(), nombre_db.lower())

        # Si la similitud está por encima del umbral y es mejor que la anterior, actualizar la mejor coincidencia
        if similitud_nombre > mejor_porcentaje:
            mejor_coincidencia = pd.concat([row_excel, row_db])
            mejor_coincidencia['Porcentaje de Similitud'] = similitud_nombre
            mejor_porcentaje = similitud_nombre

    # Agregar la mejor coincidencia a la lista de coincidencias
    if mejor_coincidencia is not None:
        coincidencias.append(mejor_coincidencia)

# Crear un DataFrame con las mejores coincidencias
coincidencias_df3 = pd.DataFrame(coincidencias)

# Función para asignar "Medico en Base" o "Medico Nuevo" en función de la similitud
def asignar_medico(row):
    return "MEDICO EN BASE" if row['Porcentaje de Similitud'] > 79 else "MEDICO NUEVO"

# Aplicar la función a cada fila y crear una nueva columna "OBSERVACION"
coincidencias_df3['OBSERVACION'] = coincidencias_df3.apply(asignar_medico, axis=1)

# Validación de cédulas
coincidencias_df3['VAL_CED'] = coincidencias_df3['CI'] == coincidencias_df3['numero_identificacion']

# Validación de especialidad
coincidencias_df3['VAL_ESP'] = coincidencias_df3['ID_ESPECIALIDAD'] == coincidencias_df3['id_especialidad']

coincidencias_df3 = coincidencias_df3.iloc[:,[0,4,10,11,12,13,14,15,16,17,18,19,20]]

coincidencias_df3.head()

,CODCUP,CI,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD,verif_ced,id_medico,NOMBRE_COMPLETO_SQL,id_especialidad,numero_identificacion,Porcentaje de Similitud,OBSERVACION,VAL_CED,VAL_ESP
0,2581014,0702884826,ALVARADO CORDOVA MARCY RODELY,23,True,14,ALVARADO CEDEÑO DORA VILMA,34,0906016357,63,MEDICO NUEVO,False,False
1,2581016,1307604478,BRAVO QUIJANO RITA ANNABEL,15,True,3122,BRAVO QUIJANO RITA ANNABEL,15,1307604478,100,MEDICO EN BASE,True,True
2,2581018,1103175459,CORDOVA PALADINES JENNY ELIZABETH,37,True,2975,HERRERA PALADINES LILIANA ELIZABETH,17,0702681693,73,MEDICO NUEVO,False,False
3,2581020,1709036824,LEMA LEMA WILSON ENRIQUE,22,True,1538,LOAYZA LOAYZA LUIS ENRIQUE,17,0700950173,68,MEDICO NUEVO,False,False
4,2581022,1804027710,MEDINA DIAZ ANABEL EMMANUELLA,23,True,593,ULLI MEDINA CARLOS EMILIANO,35,1801880681,61,MEDICO NUEVO,False,False


MODELO BERTH

In [31]:
# Cargar el modelo BERT pre-entrenado y el tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
# Crear una lista para almacenar los resultados
resultados = []
# Iterar a través de todas las filas de MED_NUEVOS2
for _, row in med_nuevos2.iterrows():
    # Obtener el texto del nombre completo en la fila actual de MED_NUEVOS2
    texto_nuevo = row['NOMBRE_COMPLETO_EXC']
    
    # Tokenizar y obtener los embeddings del texto nuevo
    inputs_nuevo = tokenizer(texto_nuevo, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings_nuevo = model(**inputs_nuevo).last_hidden_state.mean(dim=1)  # Promedio de embeddings de palabras

    # Tokenizar y obtener los embeddings de todos los elementos de SQL_DATA
    embeddings_sql_data = []
    for texto_sql in sql_data['NOMBRE_COMPLETO_SQL']:
        inputs_sql = tokenizer(texto_sql, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            embeddings_sql = model(**inputs_sql).last_hidden_state.mean(dim=1)  # Promedio de embeddings de palabras
        embeddings_sql_data.append(embeddings_sql)

    # Calcular la similitud de coseno entre el texto nuevo y todos los elementos de SQL_DATA
    similarity_scores = []
    for embedding_sql in embeddings_sql_data:
        similarity = cosine_similarity(embeddings_nuevo, embedding_sql)
        similarity_scores.append(similarity)

    # Calcular el porcentaje de similitud
    similarity_percentages = [score[0][0] for score in similarity_scores]

    # Encontrar el índice del porcentaje de similitud más alto
    max_similarity_index = similarity_percentages.index(max(similarity_percentages))

    # Obtener la fila correspondiente de SQL_DATA con el porcentaje de similitud más alto
    best_match_row = sql_data.iloc[max_similarity_index]

    # Crear un diccionario con las columnas en el orden deseado
    resultado = {
        **row.to_dict(),
        **best_match_row.to_dict(),
        'nombre_completo_exc': texto_nuevo,     
        'Similarity Percentage': max(similarity_percentages),
    }

    # Agregar el diccionario a la lista de resultados
    resultados.append(resultado)

# Crear un DataFrame con los resultados
result_df = pd.DataFrame(resultados)

# Validación de cédulas
result_df['VAL_CED'] = result_df['CI'] == result_df['numero_identificacion']

# Validación de especialidad
result_df['VAL_ESP'] = result_df['ID_ESPECIALIDAD'] == result_df['id_especialidad']


# Imprimir el DataFrame resultante
result_df.head()


,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,...,apellido_paterno,apellido_materno,nombres,numero_identificacion,id_especialidad,NOMBRE_COMPLETO_SQL,nombre_completo_exc,Similarity Percentage,VAL_CED,VAL_ESP
0,2581014,ALVARADO,CORDOVA,MARCY RODELY,0702884826,1975/01/01,MARCY.ALVARADO@HOTMAIL.COM,0993214598,1,CALLE 2 N 1256 Y CALLE 6,...,ALVARADO,RAMOS,LUPE MARGOTH,0602032930,2,ALVARADO RAMOS LUPE MARGOTH,ALVARADO CORDOVA MARCY RODELY,0.836299,False,False
1,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995/02/14,RITA.BRAVO@GMAIL.COM,0985214566,3,CALLE 3 N 1243 Y CALLE 7,...,BRAVO,QUIJANO,RITA ANNABEL,1307604478,15,BRAVO QUIJANO RITA ANNABEL,BRAVO QUIJANO RITA ANNABEL,1.000000,True,True
2,2581018,CORDOVA,PALADINES,JENNY ELIZABETH,1103175459,1980/12/25,JENNY.CORDOVA@GMAIL.COM,0965874122,1,CALLE 5 S 8554 Y CALLE 9,...,HERRERA,PALADINES,LILIANA ELIZABETH,0702681693,17,HERRERA PALADINES LILIANA ELIZABETH,CORDOVA PALADINES JENNY ELIZABETH,0.870719,False,False
3,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970/08/10,WILSON.LEMA@GMAIL.COM,0963574122,3,AVENIDA 5 N 54125 Y CALLE 63,...,TITE,AVILA,LUIS ALFREDO,0917627358,13,TITE AVILA LUIS ALFREDO,LEMA LEMA WILSON ENRIQUE,0.827230,False,False
4,2581022,MEDINA,DIAZ,ANABEL EMMANUELLA,1804027710,1991/08/06,ANABEL.MEDINA@GMAIL.COM,0996324410,1,AVENIDA 9 N 45564 Y CALLE 1,...,GALVEZ,PEREZ,MARIBEL NOEMI,1709279606,4,GALVEZ PEREZ MARIBEL NOEMI,MEDINA DIAZ ANABEL EMMANUELLA,0.836017,False,False


In [38]:
# Función para asignar "Medico en Base" o "Medico Nuevo" en función de la similitud
def asignar_medico(row):
    return "MEDICO EN BASE" if row['Similarity Percentage'] > 0.79 else "MEDICO NUEVO"

# Aplicar la función a cada fila y crear una nueva columna "OBSERVACION"
result_df['OBSERVACION'] = result_df.apply(asignar_medico, axis=1)

result_df


,CODCUP,CI,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD,verif_ced,id_medico,numero_identificacion,id_especialidad,NOMBRE_COMPLETO_SQL,nombre_completo_exc,Similarity Percentage,VAL_CED,VAL_ESP,OBSERVACION
0,2581014,0702884826,ALVARADO CORDOVA MARCY RODELY,23,True,241,0602032930,2,ALVARADO RAMOS LUPE MARGOTH,ALVARADO CORDOVA MARCY RODELY,0.836299,False,False,MEDICO EN BASE
1,2581016,1307604478,BRAVO QUIJANO RITA ANNABEL,15,True,3122,1307604478,15,BRAVO QUIJANO RITA ANNABEL,BRAVO QUIJANO RITA ANNABEL,1.000000,True,True,MEDICO EN BASE
2,2581018,1103175459,CORDOVA PALADINES JENNY ELIZABETH,37,True,2975,0702681693,17,HERRERA PALADINES LILIANA ELIZABETH,CORDOVA PALADINES JENNY ELIZABETH,0.870719,False,False,MEDICO EN BASE
3,2581020,1709036824,LEMA LEMA WILSON ENRIQUE,22,True,2944,0917627358,13,TITE AVILA LUIS ALFREDO,LEMA LEMA WILSON ENRIQUE,0.827230,False,False,MEDICO EN BASE
4,2581022,1804027710,MEDINA DIAZ ANABEL EMMANUELLA,23,True,2814,1709279606,4,GALVEZ PEREZ MARIBEL NOEMI,MEDINA DIAZ ANABEL EMMANUELLA,0.836017,False,False,MEDICO EN BASE
5,2581024,1703898567,PAEZ SANCHEZ MARTHA CECILIA,12,True,3121,1703898567,12,PEZ SANCHEZ MARTHA CECILIA,PAEZ SANCHEZ MARTHA CECILIA,0.877868,True,True,MEDICO EN BASE
6,2581026,1307584522,PIN GUADAMUD JINMY ALEXANDER,15,True,313,0913858072,21,GUZÑAY ENDARA MARY RUBY,PIN GUADAMUD JINMY ALEXANDER,0.790970,False,False,MEDICO EN BASE
7,2581020,1709036824,LEMA LEMA WILSON ENRIQUE,23,True,2944,0917627358,13,TITE AVILA LUIS ALFREDO,LEMA LEMA WILSON ENRIQUE,0.827230,False,False,MEDICO EN BASE
8,2581027,0701791634,REDROVAN TENESACA FRANCISCA EDITA,20,True,29,0701791634,20,REDROVAN TENESACA EDITA FRANCISCA,REDROVAN TENESACA FRANCISCA EDITA,0.984220,True,True,MEDICO EN BASE
9,2581029,0913837910,SALAS DAU FLORA SORAYA,16,True,68,0913837910,16,SALAS DAU FLORA SORAYA,SALAS DAU FLORA SORAYA,1.000000,True,True,MEDICO EN BASE


CONEXIÓN CON MONGODB - ALMACENAMIENTO HISTORICOS

In [39]:
# Conexión al servidor MongoDB local
client = MongoClient("localhost", 27017)

# Accede a una base de datos
db = client.laboratorio

# Convierte el DataFrame en un diccionario
registros_historicos = coincidencias_df.to_dict(orient='records')

# Obtén la fecha actual
fecha_hoy = datetime.now()

# Agregar la fecha de hoy a cada registro en el diccionario
for registro in registros_historicos:
    registro['fecha_de_ejecucion'] = fecha_hoy

# Colección en la que se almacenarán los registros históricos
historico_collection = db.historicos

# Inserta los registros en la colección de historial
historico_collection.insert_many(registros_historicos)


CONEXIÓN BBD SQL SERVER PARA INSERCIÓN DE NUEVOS MEDICOS

In [40]:
# FILTRA LA INFORMACION UNICAMENTE DE LOS MEDICOS NUEVOS QUE SE CARGARAN A LA BASE
carga_base = coincidencias_df[coincidencias_df['OBSERVACION'] == 'MEDICO NUEVO']
#Seleccionamos las columnas que se van a cargar
carga_base1 = carga_base.iloc[:, :12].copy()
carga_base1.head()

,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD
0,2581014,ALVARADO,CORDOVA,MARCY RODELY,0702884826,1975/01/01,MARCY.ALVARADO@HOTMAIL.COM,0993214598,1,CALLE 2 N 1256 Y CALLE 6,ALVARADO CORDOVA MARCY RODELY,23
2,2581018,CORDOVA,PALADINES,JENNY ELIZABETH,1103175459,1980/12/25,JENNY.CORDOVA@GMAIL.COM,0965874122,1,CALLE 5 S 8554 Y CALLE 9,CORDOVA PALADINES JENNY ELIZABETH,37
3,2581020,LEMA,LEMA,WILSON ENRIQUE,1709036824,1970/08/10,WILSON.LEMA@GMAIL.COM,0963574122,3,AVENIDA 5 N 54125 Y CALLE 63,LEMA LEMA WILSON ENRIQUE,22
4,2581022,MEDINA,DIAZ,ANABEL EMMANUELLA,1804027710,1991/08/06,ANABEL.MEDINA@GMAIL.COM,0996324410,1,AVENIDA 9 N 45564 Y CALLE 1,MEDINA DIAZ ANABEL EMMANUELLA,23
6,2581026,PIN,GUADAMUD,JINMY ALEXANDER,1307584522,1959/11/23,JINMY.PIN@GMAIL.COM,0996321447,1,AVENIDA 18 N 526 Y CALLE 632,PIN GUADAMUD JINMY ALEXANDER,15


In [41]:


# Establece la cadena de conexión a tu servidor SQL Server
server = 'BFICHASIST23'
database = 'laboratorio'
username = 'sa'
password = '1719895086'

conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};TIMEOUT=30')
cursor = conn.cursor()

for index, row in carga_base1.iterrows():
    apellido_paterno = row['APELLIDO_PAT'] if not pd.isnull(row['APELLIDO_PAT']) else None
    apellido_materno = row['APELLIDO_MAT'] if not pd.isnull(row['APELLIDO_MAT']) else None
    nombres = row['NOMBRES'] if not pd.isnull(row['NOMBRES']) else None
    id_tipo_identificacion = '1'
    numero_identificacion = row['CI']

    # Comprobación de la fecha de nacimiento
    if isinstance(row['FEC_NAC'], str):
        try:
            fecha_nacimiento = datetime.strptime(row['FEC_NAC'], '%Y/%m/%d').strftime('%Y-%m-%d')
        except ValueError as e:
            print(f"Error en la fila {index}: {e}")
            fecha_nacimiento = None
    else:
        fecha_nacimiento = row['FEC_NAC'].strftime('%Y-%m-%d') if not pd.isnull(row['FEC_NAC']) else None

    email = row['EMAIL'] if not pd.isnull(row['EMAIL']) else None
    celular = row['TEL'] if not pd.isnull(row['TEL']) else None
    direccion = row['DIRECCION'] if not pd.isnull(row['DIRECCION']) else None

    # Comprobación de id_especialidad
    id_especialidad = row['ID_ESPECIALIDAD']
    if not pd.isnull(id_especialidad):
        id_especialidad = int(id_especialidad) if isinstance(id_especialidad, (int, float)) else 0
    else:
        id_especialidad = None

    # Ejecuta la consulta
    cursor.execute("INSERT INTO dbo.medicos (apellido_paterno, apellido_materno, nombres, id_tipo_identificacion, numero_identificacion, fecha_nacimiento, email, celular, direccion, id_especialidad) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   (apellido_paterno, apellido_materno, nombres, id_tipo_identificacion, numero_identificacion, fecha_nacimiento, email, celular, direccion, id_especialidad))

# Confirma la transacción y cierra la conexión
conn.commit()
conn.close()
